# 2. Quality control and denoising
## Import Data & Packages

In [1]:
# 1 - Import all packages
import IPython
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [2]:
# 2 - Set working directory --> change this so it matches your path to the project folder
os.chdir("/home/jovyan/MicrobiomeAnalysis_TummyTribe/")

# Verify that your wroking directory is the overall project folder (.../MicrobiomeAnalysis_TummyTribe)
print("Current working directory:", os.getcwd())

Current working directory: /home/jovyan/MicrobiomeAnalysis_TummyTribe


In [3]:
# 3 - Data directories
data_in = "data/raw"
data_out = "data/preprocessing"
results_dir = "results/preprocessing"

In [4]:
# 4 - Have a look at the sequencing data
! qiime tools peek $data_in/sequences-demux-paired.qza

UUID:        b4782ab7-550b-41f5-b906-ca2cda29ca9b
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt


## Quality Control

In [11]:
! qiime demux summarize \
    --i-data $data_in/sequences-demux-paired.qza \
    --o-visualization $results_dir/raw-QC.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: results/preprocessing/raw-QC.qzv


In [5]:
Visualization.load(f"{results_dir}/raw-QC.qzv")

<visualization: Visualization uuid: 9f3ae25b-0523-414b-9f9f-69071557a076>

## Denoising - Amplicon Sequence Variants

In [15]:
! qiime dada2 denoise-paired !help

Usage: qiime dada2 denoise-paired [OPTIONS]

  This method denoises paired-end sequences, dereplicates them, and filters
  chimeras.

Inputs:
  --i-demultiplexed-seqs ARTIFACT SampleData[PairedEndSequencesWithQuality]
                          The paired-end demultiplexed sequences to be
                          denoised.                                 [required]
Parameters:
  --p-trunc-len-f INTEGER Position at which forward read sequences should be
                          truncated due to decrease in quality. This truncates
                          the 3' end of the of the input sequences, which will
                          be the bases that were sequenced in the last cycles.
                          Reads that are shorter than this value will be
                          discarded. After this parameter is applied there
                          must still be at least a 12 nucleotide overlap
                          between the forward and reverse reads. If 0 is
            

In [19]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs $data_in/sequences-demux-paired.qza \
    --p-trunc-len-f 170 \
    --p-trunc-len-r 150 \
    --p-n-threads 4 \
    --o-table $data_out/dada2_table.qza \
    --o-representative-sequences $data_out/dada2_rep_set.qza \
    --o-denoising-stats $data_out/dada2_stats.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: data/preprocessing/dada2_table.qza
Saved FeatureData[Sequence] to: data/preprocessing/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: data/preprocessing/dada2_stats.qza


In [20]:
! qiime metadata tabulate \
    --m-input-file $data_out/dada2_stats.qza \
    --o-visualization $results_dir/dada2_stats.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: results/preprocessing/dada2_stats.qzv


In [6]:
Visualization.load(f"{results_dir}/dada2_stats.qzv")

<visualization: Visualization uuid: 2e1ab96e-1f31-4e1b-b90f-48dae1215298>

### Feature Table

In [23]:
! qiime feature-table summarize \
    --i-table $data_out/dada2_table.qza \
    --m-sample-metadata-file $data_in/metadata.tsv \
    --o-visualization $results_dir/dada2_table.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: results/preprocessing/dada2_table.qzv


In [7]:
Visualization.load(f"{results_dir}/dada2_table.qzv")

<visualization: Visualization uuid: 08bca87b-d16a-47b3-b298-84e9044c9c9d>